In [1]:
import pandas as pd
import numpy as np
data = pd.DataFrame()
data['time'] = [0,0.25,0.5,0.75,1]
data['S&P500'] = np.array([1500,1530,1520.1,1575,1525])
data['LIBOR'] = [0.08,0.0835,0.08,0.085,0.082]
data

,time,S&P500,LIBOR
0,0.00,1500.0,0.0800
1,0.25,1530.0,0.0835
2,0.50,1520.1,0.0800
3,0.75,1575.0,0.0850
4,1.00,1525.0,0.0820


1. 假设甲与乙签订了 1 年期股票指数互换协议，甲支付 3 个月期美元 LIBOR，收入 S&P500 指数收益率＋0.2%。
    + 所有的互换现金流均以美元支付，每 3 个月交换一次。名义本金为 1 亿美元。试计算出乙各期的现金流？

In [17]:
from numpy import log
lag = data['S&P500'].transform(lambda x:x.shift(1))
data['sp_return'] = (data['S&P500'].diff(1)/lag).fillna(0)+0.002
data['spread'] =  data['LIBOR']/4 - data['sp_return']
data['cashflow'] = data['spread']*1e8
data

,time,S&P500,LIBOR,spread,cashflow,sp_return
0,0.00,1500.0,0.0800,0.018000,1.800000e+06,0.002000
1,0.25,1530.0,0.0835,-0.001125,-1.125000e+05,0.022000
2,0.50,1520.1,0.0800,0.024471,2.447059e+06,-0.004471
3,0.75,1575.0,0.0850,-0.016866,-1.686604e+06,0.038116
4,1.00,1525.0,0.0820,0.050246,5.024603e+06,-0.029746


2. 假设在一笔互换合约中，某一金融机构每半年支付 6 个月期的 LIBOR，同时收取 8%的年利率（半年计一次复利），名义本金为 1 亿美元。互换还有 1.25年的期限。
    + 3 个月、9 个月和 15 个月的 LIBOR（连续复利率）分别为 8%、9%和10%。上一次利息支付日的6 个月LIBOR为10 %（半年计一次复利）。
    + 试分别运用：债券组合和 FRA 组合计算此笔利率互换对该金融机构的价值。

In [26]:

import numpy as np
from numpy import e,log
libor = np.array([0.08,0.09,0.1])
prior_libor = 2*log(1+0.1/2)

#债券组合
cf = 0.5*0.08*1e8*np.ones(3)
cf[-1] = cf[-1]+1e8
time = np.array([0.25,0.75,1.25])
time_intv = 0.5
B_float = (1e8+0.1*1e8*0.5)*e**(-libor[0]*time[0])
B_fix = cf@(e**(-libor*time))
value = np.sum(B_fix-B_float)
print(f'互换价值（债券组合法）：{value}')

#FRA组合
r = 2*log(1+0.08/2)
r_spot = np.array([prior_libor,(-libor[0]*time[0]+libor[1]*time[1])/time_intv,(-libor[1]*time[1]+libor[2]*time[2])/time_intv])
FRA_value = 1e8*e**(-libor*time)*(e**(r*time_intv)-e**(r_spot*time_intv))
value = np.sum(FRA_value)
print(f'互换价值(FRA组合法):{value}')

互换价值（债券组合法）：-3481477.252720237
互换价值(FRA组合法):-3481477.2527202424


3. 一笔互换的剩余期限还有 15 个月，这一互换将年利率为 8%、本金为 2000万英镑的利息转换为年利率为 4%、本金为 3000 万美元的利息。英国与美国的利率期限结构均为水平。
    + 如果互换今天成交，互换中的美元利率为 4%，英镑为 7%，所有票面利率均为按年复利。当前即期汇率为 1.5800。
    1. 对于支付英镑的一方而言，这一互换的价值是多少？
    2. 对于支付美元的一方而言，这一互换价值又是多少？

In [5]:
import numpy as np
from numpy import e
cf_ukp,cf_usd = np.array([0.07*2e7,2e7]),np.array([0.04*2e7,2e7])
tl = np.array([0.25,1.25])
r_exchange = 1.58
ukp_long = cf_ukp@(1+0.08)**(-tl)-cf_usd@(1+0.04)**(-tl)*r_exchange
print(f'英镑多头价值：{ukp_long}英镑')
usd_long = -cf_ukp@(1+0.08)**(-tl)/r_exchange+cf_usd@(1+0.04)**(-tl)
print(f'美元多头价值：{usd_long}美元')

英镑多头价值：-11800867.180399578英镑
美元多头价值：7468903.278733909美元


4. 公司 A希望以浮动利率借入美元，公司 B想以固定利率借入加元。一家金融机构计划安排一个货币互换，并想从中盈利 50 个基点。

    \begin{matrix}
    0 & A & B \\
    美元(浮动) & LIBOR+0.5\% & LIBOR+1.0\% \\
    加元(固定) & 5\% & 6.5\%
    \end{matrix}
    
    + 如果这一互换对于A 和 B 有同样的吸引力，由金融机构承担汇率风险，A 和 B 最终支付的利率分别为多少？
    + 请设计一个利率互换

1. 不合作总成本：6.5%＋LIBOR＋0.5%＝LIBOR＋7%
2. 合作总成本：5%+LIBOR+1%＝LIBOR＋6%
3. 合作的总收益：1%-0.5% = 0.5%
4. 假设双方不承担汇率风险：
    + A的筹资成本：LIBOR+0.5%-0.25%=(LIBOR+0.25%)USD
    + B的筹资成本：6.5%-0.25%=(6.25%)CAD
    
5. $ {}^{5\% CAD}\leftarrow A {}^{5\% CAD}\rightleftarrows^{LIBOR+0.25\% USD} Bank {}^{6.25\%CAD}\rightleftarrows^{LIBOR\space USD} B \rightarrow^{LIBOR+1\% USD}  $
